In [ ]:
!pip install xgboost --user

In [ ]:
%matplotlib inline
from joblib import dump, load
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

# Models
from sklearn.ensemble import BaggingRegressor as BR, RandomForestRegressor as RFR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.model_selection import GridSearchCV, cross_val_score, ParameterGrid

from xgboost import XGBRegressor as XGBR

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Metrics
from sklearn.metrics import r2_score, mean_squared_error

TRAIN_SMALL_PATH = "./etc/training_small.csv"
TRAIN_LARGE_PATH = "./etc/training_large.csv"
TEST_PATH = "./etc/test.csv"
TARGET = "price"

HIGH_RELEVANCE_COLS = ['price', 'city_fuel_economy', 'engine_displacement', 'horsepower', 'highway_fuel_economy', 'year', 'mileage', 'owner_count', 'make_name', 'body_type']
NUMERICAL_COLS = ["price", 'city_fuel_economy', 'engine_displacement', 'horsepower', 'highway_fuel_economy', 'year', "mileage", "owner_count"]
CATEGORICAL_COLS = ["make_name", 'body_type'] #, 'frame_damaged' "model_name" ]

BAGGING_PARAMS = {
    'n_estimators': np.arange(10, 101, 10),  # Number of base estimators in the ensemble
    'max_samples': np.arange(0.1, 1.1, 0.1),  # The number of samples to draw from X to train each base estimator
    'max_features': np.arange(0.1, 1.1, 0.1),  # The number of features to draw from X to train each base estimator
    'bootstrap': [True, False],  # Whether samples are drawn with replacement
    # 'bootstrap_features': [True, False],  # Whether features are drawn with replacement
    'oob_score': [True, False],  # Whether to use out-of-bag samples to estimate the generalization error
}

RFR_PARAMS = {
    'n_estimators': np.arange(10, 101, 10),  # Number of trees in the forest
    'max_depth': np.arange(3, 17, 2).tolist() + [None],  # Maximum depth of the tree
    'min_samples_split': np.arange(2, 11, 2),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': np.arange(1, 11, 2),  # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2', 1.0],  # Number of features to consider when looking for the best split
    # 'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
}

XGBR_PARAMS = {
    'n_estimators': np.arange(10, 101, 10),  # Number of gradient boosted trees
    'learning_rate': [0.001, 0.01, 0.05],  # Step size shrinkage to prevent overfitting
    'max_depth': np.arange(3, 11, 2),  # Maximum depth of a tree
    # 'min_child_weight': np.arange(1, 6, 1),  # Minimum sum of instance weight (hessian) needed in a child
    'gamma': [i/10.0 for i in range(0, 5)],  # Minimum loss reduction required to make a further partition
    'subsample': [i/10.0 for i in range(6, 11)],  # Fraction of samples used per tree
    # 'colsample_bytree': [i/10.0 for i in range(6, 11)],  # Fraction of features used per tree
    # 'scale_pos_weight': [1, 10, 25, 50, 75, 99, 100],  # Control the balance of positive and negative weights
    'reg_alpha': [1e-2, 0.1, 1, 100],  # L1 regularization term on weights
    # 'reg_lambda': [1e-2, 0.1, 1, 100],  # L2 regularization term on weights
}

# Preprocessing

In [ ]:
def label_encode(dataFrame, cols) -> pd.DataFrame:
    les = {}
    for i, col in enumerate(cols):
        le = LabelEncoder()
        dataFrame[col] = le.fit_transform(dataFrame[col])
        les[col] = le
    return les

def preprocess(df: pd.DataFrame, testing: bool = False, les = None):
    # Create a copy of the DataFrame
    X = df.copy()

    # Select only a few columns
    X = X[HIGH_RELEVANCE_COLS]
    
    if (testing):
        y = None
    else:
        # Get the target
        y = X["price"].apply(lambda x: math.log(x))

    # Drop the "price" column because we don't need it anymore
    X.drop("price", axis=1, inplace=True)
    
    # Feature engineering part 1
    # Group 'owner_count' >= 4 into 4
    X['owner_count'] = X['owner_count'].apply(lambda x: 4 if x >= 4 else x)
    
    # Transformers: I use different transformers for numerical and categorical columns.
    numerical_transformer = Pipeline([
        ('scaler', MinMaxScaler()),
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median'))
    ])
    
    les = label_encode(X, CATEGORICAL_COLS)
    
    # Define a simplified ColumnTransformer with a single transformer
    # The steps of this pipeline are as follows:
    # 1. Scaling: Scales values according to the N(0, 1) distribution
    # 2. Impute: Deals with NaNs by imputing them with the mean
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, NUMERICAL_COLS[1:]), # Don't include "price"
            ('cat', categorical_transformer, CATEGORICAL_COLS)
        ])
    
    # Create a pipeline
    pipe = Pipeline([('preprocessor', preprocessor)])
    
     # And fit it
    X = pd.DataFrame(pipe.fit_transform(X), columns=X.columns)
    
    # Feature engineering part 2
    X['avg_economy'] = (X['city_fuel_economy'] + X['highway_fuel_economy']) / 2    
    X['engine_vol_vs_hp'] = X['engine_displacement'] / X['horsepower']
    
    # Deal with infinities
    X['engine_vol_vs_hp'].replace([np.inf, -np.inf], 0, inplace=True)
    
    # Drop columns
    X.drop(['city_fuel_economy', 'highway_fuel_economy', 'engine_displacement', 'horsepower'], axis=1, inplace=True)
    
    if (not testing):
        # Convert the labels to integers
        for col in CATEGORICAL_COLS:
                X[col] = X[col].apply(lambda x: int(x))

    return X, y, les

def train_and_evaluate(model, X_train, y_train, X_test, y_test, params: dict):    
    # Use GridSearchCV with the maximum number of CPU cores available
    grid_search = GridSearchCV(model, params, cv=3, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

    print("Starting grid search: ", grid_search)
    
    # Train the model using GridSearchCV
    grid_search.fit(X_train, y_train)
    
    print("Grid search complete.")

    # Best estimator after grid search
    best_model = grid_search.best_estimator_

    # Predict using the best model
    y_pred = best_model.predict(X_test)

    # Evaluate
    r_squared = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Get cross-validation scores from GridSearchCV results
    cv_scores = grid_search.cv_results_['mean_test_score']
    
    return {
        'cv_scores': cv_scores,
        'r_squared': r_squared,
        'mse': mse,
        'y_pred': y_pred,
        'model': best_model
    }

def plot_residuals(actual_values, predicted_values, filename: str):
    residuals = actual_values - predicted_values
    
    skew = stats.skew(residuals)
    kurtosis = stats.kurtosis(residuals)
    print("Skew: ", skew)
    print("Kurtosis: ", kurtosis)
    
    # Set up the subplot layout
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))

    # Residuals Plot
    ax[0].scatter(predicted_values, residuals, alpha=0.6)
    ax[0].axhline(y=0, color='r', linestyle='--')
    ax[0].set_xlabel('Predicted Values')
    ax[0].set_ylabel('Residuals')
    ax[0].set_title('Residuals vs. Predicted Values')

    # Q-Q plot
    stats.probplot(residuals, dist="norm", plot=ax[1])
    ax[1].set_title('Q-Q plot of the Residuals')

    plt.tight_layout()
    plt.savefig(f"./img/{filename}_residuals.png")
    plt.show()
    
    return skew, kurtosis

def plot_actual_vs_predicted(actual_values, predicted_values, filename: str, title: str ='Predicted vs Actual'):
    """
    Plot actual values against predicted values in a scatter plot.
    
    Parameters:
    - actual_values: Actual target values.
    - predicted_values: Predicted values from the model.
    - title: Title of the plot.
    
    Returns:
    - A scatter plot of actual vs. predicted values.
    """    
    plt.figure(figsize=(8, 6))
    plt.scatter(actual_values, predicted_values, alpha=0.5)
    plt.plot([min(actual_values), max(actual_values)], [min(actual_values), max(actual_values)], color='red')  # Diagonal line
    plt.title(title)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.grid(True)
    plt.savefig(f"./img/{filename}.png")
    plt.show()

def get_test_preds(model, les, model_name, large: bool = False):
    # Load test data
    test = pd.read_csv(TEST_PATH)
    
    # Preprocess test data
    test_preprocessed, _, _ = preprocess(test, True, les)
    
    print(test_preprocessed.head())
    
    # Get predictions
    print("Predictions:")
    preds = model.predict(test_preprocessed)
    df = pd.DataFrame(preds)
    print(df.head())

    # And save them
    filename = f"./etc/test_preds_{model_name}.csv"
    np.savetxt(filename, preds, delimiter=",", header=TARGET)

    # Sanity check
    d = pd.read_csv(filename)
    print(d.shape)
    print(d.head())
    
    return preds

def master(model_name: str, large: bool = False):
    '''Master method to do everything.'''
    
    # Select the model and the parameters
    model = None
    gcv_params = None
    if (model_name == "BR"):
        model = BR
        gcv_params = BAGGING_PARAMS
    elif (model_name == "RFR"):
        model = RFR
        gcv_params = RFR_PARAMS
    elif (model_name == "XGBR"):
        model = XGBR
        gcv_params = XGBR_PARAMS
        
    if (model is None or gcv_params is None):
        raise Exception("Incorrect parameters in `master()`.")
    
    # Select the dataset
    dataset = TRAIN_SMALL_PATH
    d = "small"
    if (large):
        dataset = TRAIN_LARGE_PATH
        d = "large"
    
    print("Dataset: ", dataset)
    train = pd.read_csv(dataset)
    
    print("------------------------------------")
    
    # Preprocess the dataset
    X, y, les = preprocess(train)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1203)
    print("X_train.tail(2):")
    print(X_train.tail(2))
    
    print("------------------------------------")
    print(f"Training {model_name}...")
    
    results = train_and_evaluate(model(), X_train, y_train, X_test, y_test, gcv_params)

    # Extract the results
    cv_scores = results['cv_scores']
    r_squared = results['r_squared']
    mse = results['mse']
    best_model = results['model']
    
    name = f"{model_name}_{d}"

    # Print or use the extracted results as needed
    # print(f'Cross-Validation Scores: {cv_scores}')
    print(f'R squared: {r_squared}')
    print(f'MSE: {mse}')
    print(f'Best Model: {best_model}')
    
    print("------------------------------------")
    print("Plotting...")
    y_preds = best_model.predict(X_test)
    plot_actual_vs_predicted(y_test, y_preds, name, "Predicted vs Actual")
    skew, kurtosis = plot_residuals(y_test, y_preds, f"{name}_residuals")
    
    results['skew'] = skew
    results['kurtosis'] = kurtosis
    
    # Save the model
    print("Saving model...")
    dump(best_model, filename=name)
    print("Saved.")
    
    
    print("------------------------------------")
    print("Testing...")
    get_test_preds(best_model, les, name, large)
    
    return results

# Using the small dataset

## Bagging

In [ ]:
br_results_small = master("BR", False)

## Random forests

In [ ]:
rfr_results_small = master("RFR", False)

## (Extreme Gradient) Boosting

In [ ]:
xgbr_results_small = master("XGBR", False)

# Using the large dataset

## Bagging

In [ ]:
br_results_large = master("BR", True)

## Random Forests

In [ ]:
rfr_results_large = master("RFR", True)

## (Extreme Gradient) Boosting

In [ ]:
xgbr_results_large = master("XGBR", True)